# COGS 108 - Final Project 

# Overview

The research conducted focuses on the effect of time on the karma genreated for a given post on Reddit. We investigate the optimal time of day, and day of the week to post on 'r/funny' that would generate the greatest amount of upvotes, comments and views that readily predict a high karma count. The findinds of this research question are useful beyond the trivial matter of boosting karma, they are valuable for businesses that use Reddit for marketing, for election posts that desire the greatest audience, for the greatest exposure to breaking news and controversial matters, etc. The implications of the research question at hand are great and can be further applied when additional subreddits are integrated. 

Our findings suggest that the optimal time to post for a high karma count are:..

# Names

- Cyrus Shen
- Randy Vo
- Kimberly Chuong
- Josue Ayala
- Conner Henderson
- Mykel Rodriguez

# Group Members IDs

- A########
- A########
- A########
- A########
- A########

# Research Question

What day of the week and time of day is optimal for maximizing the amount of karma for a given post on the r/funny subreddit of Reddit?

## Background and Prior Work

To run analysis on reddit data, we first had to gain permission to use Reddit's API. The API is readily accessibe after registering for use, and is easily manipulated using the Reddit API wrapper, Python praw. 
References (include links):
- 1)
- 2)

# Hypothesis


We predict that the weekends will be more effective for karma than weekdays and that evenings will be more effective than mornings and afternoons. Specificlaly, we predict that Saturday at 9pm will have the most posts overall with the highest upvotes and comments. 

# Dataset(s)

*Fill in your dataset information here*

(Copy this information for each dataset)
- Dataset Name:
- Link to the dataset:
- Number of observations:

1-2 sentences describing each dataset. 

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.

# Setup

In [3]:
## YOUR CODE HERE

# Data Cleaning

Describe your data cleaning steps here.

In [4]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION

# Data Analysis & Results

Include cells that describe the steps in your data analysis.

In [5]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION

# Ethics & Privacy

   As discussed in the introduction of our research, the implicaitons of this data is great. Although there are many limitations in the methods of our data collection process, the concept of this research question is very important. Using the data from this research in other subreddits could lead to more serious effects on post interaction. Users could utilize these findings to strategically generate the most karma per post, click-farms can capitilize on this time efficacy, companies can maximize the monetary value of their marketing posts, and unfortunately, hategroups or negative propganda posts can also rely on this data for maximum viewer exposure and sharing potential.

   Additionally, although we do not collect any personally identifying information, the data collected is user generated and therefore may lead to a loss of confidentiality in ways we cannot forsee. For example, one issue we encountered was when we collected the current online active subscribers of r/funny; these username identifications are mostly annonymous but can inherently contain personally identifying aspects. Furthermore, if these research methods are expanded to mutiple subreddits, users can be cross idenified as being active in various subreddits that are personally identifying that can very well lead to loss of annomitiy. 

# Conclusion & Discussion

*Fill in your discussion information here*